## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements (*Note: ocdskingfishercolab installs google-colab, which expects specific versions of pandas and numpy*):


In [ ]:
! pip install --upgrade pip > pip.log
! pip install --upgrade 'ocdskingfishercolab<0.4' ipywidgets psycopg2-binary >> pip.log

In [ ]:
# @title Import packages and load extensions { display-mode: "form" }

import gzip
import json
import os
import shutil
import tempfile
from collections import Counter
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from google.colab.data_table import DataTable
from google.colab.files import download
from ipywidgets import widgets
from ocdskingfishercolab import (
    authenticate_gspread,
    calculate_coverage,
    download_dataframe_as_csv,
    format_thousands,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
    set_spreadsheet_name,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

In [ ]:
# @title Configure the notebook environment { display-mode: "form" }

# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()

## Charts Setup
*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

In [ ]:
! pip install altair pyarrow==14.0.0 >> pip.log

Import chart packages and define chart functions. The currently available chart functions are:

* Release count
* Objects per stage
* Releases by month
* Objects per year
* Top buyers
* Usability indicators

In [ ]:
# @title Chart functions { display-mode: "form" }
import altair as alt


class MissingColumnsError(Exception):
    def __init__(self, columns):
        super().__init__(f"The source data is missing one or more of these columns: {columns}")


chart_properties = {
    "width": 600,
    "height": 350,
    "padding": 50,
    "title": alt.TitleParams(text="", subtitle=[""], fontSize=18),
}
chart_axis = {
    "titleFontSize": 14,
    "labelFontSize": 14,
    "labelPadding": 5,
    "ticks": False,
    "domain": False,
}


def check_columns(columns, data):
    # check if input contains the right columns
    if not columns.issubset(data.columns):
        raise MissingColumnsError(columns)


def plot_release_count(release_counts):
    check_columns({"collection_id", "release_type", "release_count", "ocid_count"}, release_counts)
    return (
        alt.Chart(release_counts)
        .mark_bar()
        .encode(
            x=alt.X(
                "release_count",
                type="ordinal",
                axis=alt.Axis(title="release count", labelAngle=0),
            ),
            y=alt.Y(
                "ocid_count",
                type="quantitative",
                axis=alt.Axis(title="ocid count", format="~s", tickCount=5),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                title="release type",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
            ),
            tooltip=[
                alt.Tooltip("release_count", title="release count"),
                alt.Tooltip("ocid_count", title="ocid count", format="~s"),
                alt.Tooltip("release_type", title="release type"),
                alt.Tooltip("collection_id", title="collection id"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_objects_per_stage(objects_per_stage):
    check_columns({"stage", "object_count"}, objects_per_stage)
    stages = ["planning", "tender", "awards", "contracts", "implementation"]
    return (
        alt.Chart(objects_per_stage)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "stage",
                type="ordinal",
                scale=alt.Scale(domain=stages),
                sort=stages,
                axis=alt.Axis(title="stage", labelAngle=0),
            ),
            y=alt.Y(
                "object_count",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=len(stages)),
            ),
            tooltip=[
                alt.Tooltip("stage", title="stage"),
                alt.Tooltip("object_count", title="number of objects"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_releases_by_month(release_dates):
    check_columns({"date", "collection_id", "release_type", "release_count"}, release_dates)
    max_rows = 5000
    # check if number of rows is more than 5000
    if release_dates.shape[0] > max_rows:
        alt.data_transformers.disable_max_rows()

    # draw chart
    return (
        alt.Chart(release_dates)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X("date", timeUnit="yearmonth", axis=alt.Axis(title="year and month")),
            y=alt.Y(
                "release_count",
                type="quantitative",
                aggregate="sum",
                axis=alt.Axis(title="number of releases", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
                legend=alt.Legend(title="release type"),
            ),
            tooltip=[
                alt.Tooltip("date", timeUnit="yearmonth", title="date"),
                alt.Tooltip("release_count", aggregate="sum", title="number of releases"),
                alt.Tooltip("release_type", title="release type"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_objects_per_year(objects_per_year):
    check_columns({"year", "tenders", "awards"}, objects_per_year)
    stages = ["tenders", "awards"]
    return (
        alt.Chart(objects_per_year)
        .transform_fold(stages)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X(
                "year",
                type="quantitative",
                axis=alt.Axis(title="year", format=".0f", tickCount=objects_per_year.shape[0]),
            ),
            y=alt.Y(
                "value",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "key",
                type="nominal",
                title="object type",
                scale=alt.Scale(domain=stages, range=["#D6E100", "#FB6045"]),
            ),
            tooltip=[
                alt.Tooltip("year", title="year", type="quantitative"),
                alt.Tooltip("value", title="number of objects", type="quantitative"),
                alt.Tooltip("key", title="object type", type="nominal"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_top_buyers(buyers):
    check_columns({"name", "total_tenders"}, buyers)
    return (
        alt.Chart(buyers)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "total_tenders",
                type="quantitative",
                axis=alt.Axis(title="number of tenders", format="~s", tickCount=5),
            ),
            y=alt.Y(
                "name",
                type="ordinal",
                axis=alt.Axis(title="buyer", labelAngle=0),
                sort=alt.SortField("total_tenders", order="descending"),
            ),
            tooltip=[
                alt.Tooltip("name", title="buyer", type="nominal"),
                alt.Tooltip("total_tenders", title="number of tenders", type="quantitative"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )


def plot_usability_indicators(data, lang="English"):
    labels = {
        "English": {
            "nrow": "row_number(indicator)",
            "sort": "calculation",
            "y_sort": "indicator",
            "groupby": "Use case",
            "title": "number of indicators",
            "tooltip_missing": "Missing Fields",
        },
        "Spanish": {
            "nrow": "row_number(Indicador)",
            "sort": "¿Se puede calcular?",
            "y_sort": "Indicador",
            "groupby": "Caso de Uso",
            "title": "Número de indicadores",
            "tooltip_missing": "Campos faltantes",
        },
    }
    return (
        alt.Chart(data)
        .transform_window(
            nrow=labels[lang]["nrow"],
            frame=[None, None],
            sort=[{"field": labels[lang]["sort"]}],
            groupby=[labels[lang]["groupby"]],
        )
        .mark_circle(size=250, opacity=1)
        .encode(
            x=alt.X(
                "nrow",
                type="quantitative",
                axis=alt.Axis(title=[labels[lang]["title"], ""], orient="top", tickCount=5),
            ),
            y=alt.Y(
                labels[lang]["groupby"],
                type="nominal",
                sort=alt.Sort(field=labels[lang]["y_sort"], op="count", order="descending"),
            ),
            color=alt.Color(
                labels[lang]["sort"],
                type="nominal",
                scale=alt.Scale(range=["#fb6045", "#d6e100"]),
                legend=alt.Legend(title=[labels[lang]["sort"]]),
            ),
            tooltip=[
                alt.Tooltip(labels[lang]["y_sort"], type="nominal"),
                alt.Tooltip(labels[lang]["groupby"], type="nominal"),
                alt.Tooltip(labels[lang]["sort"], type="nominal"),
                alt.Tooltip(labels[lang]["tooltip_missing"], type="nominal"),
            ],
        )
        .properties(**chart_properties)
        .configure_axis(**chart_axis)
        .configure_view(strokeWidth=0)
    )

## Setup Kingfisher Process

### Connect to the database

In [ ]:
import getpass

from ocdskingfishercolab import (
    list_collections,
    list_source_ids,
    set_search_path,
)

Enter your PostgreSQL credentials and connect to the Kingfisher Process database:

In [ ]:
user = input("Username:")
password = getpass.getpass("Password:")

# Don't show connection string after execute.
%config SqlMagic.displaycon = False

connection_string = (
    "postgresql://"
    + user
    + ":"
    + password
    + "@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require"
)
%sql $connection_string

### Choose collections and schema

*Use this section to choose the collections and schema that you want to query.*

#### Set the collection(s)

Update `collection_ids` with the `id`(s) of the [Kingfisher Process collection(s)](https://kingfisher-process.readthedocs.io/en/latest/data-model.html#collections):

In [ ]:
collection_ids = (2358, 2359)

If you don't know which collections you need, run the next cell and use the **Filter** button to filter the [collection table](https://kingfisher-process.readthedocs.io/en/latest/database-structure.html#collection-table) to find the collection(s). You can use the `source_id` column to filter on the `name` of the [Kingfisher Collect spider](https://kingfisher-collect.readthedocs.io/en/latest/spiders.html) used to collect the data. Use the value(s) from the `id` column to update the previous cell.

In [ ]:
list_collections()

#### Set the schema

Update `schema_name` with the name of the [Kingfisher Summarize schema](https://kingfisher-summarize.readthedocs.io/en/latest/index.html#how-it-works).

In [ ]:
schema_name = "view_data_collection_2358_2359"
set_search_path(schema_name)

If you don't know which schema you need, run the next cell and use the **Filter** button to filter the [selected collections table](https://kingfisher-summarize.readthedocs.io/en/latest/database.html#summaries-selected-collections) to find the schema. You can use the `collection_id` column to filter on the `id` of the collections that you identified in the previous step. Alternatively, you can filter on the `source_id` column. Use the value from the `schema` column to update the previous cell.

In [ ]:
%%sql
SELECT
    summaries.selected_collections.*,
    source_id
FROM
    summaries.selected_collections
INNER JOIN
    collection
    ON summaries.selected_collections.collection_id = collection.id


If you can't find a schema containing the collections that you want to query, you can create a schema using [Kingfisher Summarize](https://ocdsdeploy.readthedocs.io/en/latest/use/kingfisher-summarize.html).

## Check for data collection and processing errors

Kingfisher Collect and Kingfisher Process log messages that might indicate OCDS implementation errors or API stability issues.

Confirm any errors or warnings by manually checking the OCDS publication.

### Kingfisher Collect

See [how to review the Kingfisher Collect crawl’s log file](https://kingfisher-collect.readthedocs.io/en/latest/logs.html).

**Note:** There is an open [pull request](https://github.com/open-contracting/notebooks-ocds/pull/44) to automate this. If interested, please comment on the pull request to prioritize it.

### Kingfisher Process

Users add a note when starting a crawl and when running the `load` command.

Kingfisher Process adds notes (the `note` column) at different levels (the `code` column):

- `INFO`
  - *load*: The Kingfisher Collect crawl's reason for closing and statistics (the `data` column).
- `WARNING`
  - *compile*: When a record has undated releases, linked releases or no releases, but a compiled release can be calculated (by merging remaining dated releases, using the `compiledRelease` field, or using an undated release with a 'compiled' tag).
- `ERROR`
  - *load*: When the input data is invalid JSON.
  - *compile*: When no compiled release can be calculated for a record.

📗 **You can check the full list of note types and what they mean in the Kingfisher Process's [documentation](https://kingfisher-process.readthedocs.io/en/latest/database.html#collection-note-table)**📗


List the notes for each of your collections:

In [ ]:
%%sql
SELECT
    collection_id,
    code,
    note,
    data
FROM
    collection_note
WHERE
    collection_id IN :collection_ids


## Perform manual conformance checks

[Conforming publications](https://standard.open-contracting.org/latest/en/schema/conformance_and_extensions/#publication-conformance):

* Use a [registered OCID prefix](https://standard.open-contracting.org/latest/en/schema/identifiers/#contracting-process-identifier-ocid).
* The [OCDS Data Review Tool](https://standard.open-contracting.org/review/) can report results on the data.
* Must not use terms from outside the OCDS schema where its terms would suffice.

Use this section to:

* Check if the proper OCID prefix is in use.
* Check if the OCDS Data Review Tool is able to report results on data.
* Identify fields in local extensions and additional fields and codes that should be mapped to fields and codes in the OCDS schema and extensions.
* Identify deprecated fields, in order to recommend a change to the publisher's OCDS implementation.


You can use the following resources to find fields and codes with similar semantics:

* [OCDS Schema and Codelist Reference](https://standard.open-contracting.org/latest/en/schema/), for fields in the core OCDS schema.
* [OCDS Extensions Field and Code Search](https://open-contracting.github.io/editor-tools/), for fields and codes in OCDS extensions.
* [GitHub Issue Tracker](https://github.com/open-contracting/standard/issues), for discussions about adding new fields and codes.

Check that field and code names [conform to the style guide](https://ocds-standard-development-handbook.readthedocs.io/en/latest/meta/schema_style_guide.html#field-and-code-names) and report any issues to the publisher.

If you cannot find a suitable mapping for an additional field or code, [open a GitHub issue](https://github.com/open-contracting/standard/issues) to describe the semantics of the field or code and to discuss how to model it. Report any issues to the publisher.

**Note:** This section depends on the `check` step of Kingfisher Process having completed. See *Check for structure and format errors >Confirm that checks are complete*. If the checks are not complete or the OCDS Data Review Tool is not able to report results on data, **the publisher don't pass that conformance criterion**

### OCID prefix

Check that the data uses the OCID prefix that was issued to this specific publisher.

You only need to run either the *Release prefixes* or *Record prefixes* section, depending on the publication's data format.

Update the ocid prefix in the appropriate cell. Prefixes can be found at [the list of registered prefixes](https://docs.google.com/spreadsheets/d/1E5ZVhc8VhGOakCq4GegvkyFYT974QQb-sSjvOfaxH7s/pubhtml?gid=506986894&single=true&widget=true).

#### **Release prefixes**

Notify the publisher of any incorrect prefixes.

In [ ]:
# Do not remove the final % character
ocid_prefix = "%"

In [ ]:
%%sql ocid_prefix_release_check <<
SELECT ocid
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND ocid NOT LIKE :ocid_prefix


In [ ]:
ocid_prefix_release_check

#### **Record prefixes**

Notify the publisher of any incorrect prefixes.

In [ ]:
%%sql ocid_prefix_record_check <<
SELECT ocid
FROM
    record_summary
WHERE
    collection_id IN :collection_ids
    AND ocid NOT LIKE :ocid_prefix


In [ ]:
ocid_prefix_record_check

### Local extensions

For each field and code in extensions authored by the publisher, in addition to the above checks, consider whether to [review the extension in detail](https://docs.google.com/document/d/1CS_TMubqoYaucT8JXPTgLS-mF4eMIifX-6mD0xpWg9M/edit).

List the extensions declared in the package metadata.

**Note:** This query should be kept in sync with the query in *Check scope > Extensions*.


In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    jsonb_array_elements(package_data -> 'extensions') AS ocds_extension,
    count(*) AS count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND package_data IS NOT NULL
GROUP BY
    collection_id,
    release_type,
    ocds_extension
ORDER BY
    collection_id ASC,
    release_type ASC,
    count DESC;


### Additional fields

[lib-cove-ocds](https://github.com/open-contracting/lib-cove-ocds) reports additional fields in the following scenarios:

* Fields from undeclared extensions.
* Fields with language variations, e.g. `title_es`. You do not need to report language variations to the publisher, but you should check that the field [conforms to the rules for language variations](https://standard.open-contracting.org/latest/en/schema/reference/#language).
* OCDS 1.0 data using extension fields. You should report the fields to the publisher and recommend that they upgrade to OCDS 1.1.


List additional fields.

By default, results are reported for a sample of 10% of releases. For small collections, you can set `sample_size` to `1` to return results for the full collection. For large collections, you can reduce the sample size.

In [ ]:
sample_size = 0.1

In [ ]:
%%sql
WITH check_results AS (
    SELECT
        *,
        CASE
            WHEN (release_type IN ('record', 'embedded_release'))
                THEN
                    record_check
            ELSE
                release_check
        END AS results
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type <> 'compiled_release'
        AND random() < :sample_size
),

counts AS (
    SELECT
        collection_id,
        release_type,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field,
        sum((additional_fields ->> 'count')::int) AS count
    FROM
        check_results
    CROSS JOIN
        jsonb_array_elements(
            results -> 'all_additional_fields'
        ) AS additional_fields
    GROUP BY
        collection_id,
        release_type,
        field,
        path
    ORDER BY
        path ASC,
        count DESC
),

examples AS (
    SELECT DISTINCT ON (
        collection_id,
        release_type,
        additional_fields ->> 'path',
        additional_fields ->> 'field_name')
        collection_id,
        release_type,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field,
        additional_fields ->> 'examples' AS examples
    FROM
        check_results
    CROSS JOIN
        jsonb_array_elements(
            results -> 'all_additional_fields'
        ) AS additional_fields
    WHERE
        jsonb_array_length(additional_fields -> 'examples') > 0
)

SELECT
    counts.collection_id,
    counts.release_type,
    counts.path,
    counts.field,
    count,
    examples
FROM
    counts
LEFT JOIN examples
    USING (
        collection_id,
        release_type,
        path,
        field
    )
ORDER BY
    path,
    field;


#### Additional field examples

Generate a release package containing an example release for each additional field:

In [ ]:
%%sql additional_field_examples <<
WITH additional_field_releases AS (
    SELECT
        ocid,
        release.release_id,
        data_id,
        additional_fields ->> 'path' AS path,
        additional_fields ->> 'field_name' AS field
    FROM
        release_check
    CROSS JOIN
        jsonb_array_elements(
            cove_output -> 'all_additional_fields'
        ) AS additional_fields
    INNER JOIN release ON release_check.release_id = release.id
    WHERE
        collection_id IN :collection_ids
        AND random() < :sample_size
),

additional_fields AS (
    SELECT DISTINCT
        path,
        field
    FROM
        additional_field_releases
),

examples AS (
    SELECT DISTINCT ON (
        additional_fields.path,
        additional_fields.field)
        additional_fields.path,
        additional_fields.field,
        ocid,
        release_id,
        data_id,
        data
    FROM
        additional_fields
    INNER JOIN additional_field_releases
        ON
            additional_fields.path = additional_field_releases.path
            AND additional_fields.field = additional_field_releases.field
    INNER JOIN data ON data.id = data_id
    ORDER BY
        additional_fields.path,
        additional_fields.field
)

SELECT jsonb_build_object('releases', jsonb_agg(data)) AS release_package
FROM
    examples


In [ ]:
render_json(additional_field_examples["release_package"][0])

### Additional codes

List additional codes in the context of an open codelist.

Using additional codes in the context of a closed codelist is an error, and is reported in the *Check for structure and format errors* section.

In [ ]:
%%sql
WITH check_results AS (
    SELECT
        *,
        CASE
            WHEN (release_type IN ('record', 'embedded_release'))
                THEN
                    record_check
            ELSE
                release_check
        END AS results
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type <> 'compiled_release'
)

SELECT
    collection_id,
    release_type,
    additional_open_codelist_values.value -> 'codelist' AS codelist,
    codes.value AS code,
    count(*) AS occurrences
FROM
    check_results
CROSS JOIN
    jsonb_each(
        results -> 'additional_open_codelist_values'
    ) AS additional_open_codelist_values
CROSS JOIN jsonb_array_elements(value -> 'values') AS codes
GROUP BY
    collection_id,
    release_type,
    codelist,
    code
ORDER BY
    collection_id,
    release_type,
    codelist,
    count(*) DESC


### Deprecated fields

Before a field or codelist is removed from the standard, it is first marked as [deprecated](https://standard.open-contracting.org/latest/en/governance/deprecation/#deprecation).

Use this section to check for deprecated fields.

List deprecated fields:

In [ ]:
%%sql
SELECT DISTINCT ON (collection_id, path, deprecated_version, explanation)
    collection_id,
    regexp_replace(trim('"' FROM paths::text), '\/[0-9]+', '', 'g')
    || '/'
    || (deprecated_fields ->> 'field') AS path,
    deprecated_fields -> 'explanation' -> 0 AS deprecated_version,
    deprecated_fields -> 'explanation' -> 1 AS explanation,
    ocid AS example_ocid
FROM
    release_check
CROSS JOIN
    jsonb_array_elements(
        cove_output -> 'deprecated_fields'
    ) AS deprecated_fields
CROSS JOIN jsonb_array_elements(deprecated_fields -> 'paths') AS paths
INNER JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids;


## Check scope



Use this section to check:

* how many releases, records and compiled releases your data contains
* what stages of the contracting process your data covers
* what date range your data covers

If you are preparing an [Ad-hoc structure and format feedback](https://docs.google.com/document/d/1_k7eA2rI-k5EH8VESkVAB73wa_qrpplL-7dKgMLTGZc/edit#heading=h.i7tpu8c49dcv), you might skip this section.

### Release and record counts

Collections in Kingfisher Process contain either [releases](https://standard.open-contracting.org/latest/en/schema/reference/), [records](https://standard.open-contracting.org/latest/en/schema/records_reference/) or [compiled releases](https://standard.open-contracting.org/latest/en/schema/records_reference/#compiled-release). Kingfisher Process creates compiled release collections from release or record collections.

Use this section to check that the data contains the expected number of releases, records and compiled releases. Where possible, you should check these numbers against the total number of results available in the frontend of the data source.

Count the number of releases, records and compiled releases, for each collection.

**Note:** These columns are not yet populated in version 2 of Kingfisher Process. Comment on [this issue](https://github.com/open-contracting/kingfisher-process/issues/370) to prioritize it.

In [ ]:
%%sql
SELECT
    id AS collection_id,
    cached_releases_count AS releases_count,
    cached_records_count AS records_count,
    cached_compiled_releases_count AS compiled_releases_count
FROM
    collection
WHERE
    id IN :collection_ids


### Contracting process stages

Use this section to check that the data covers the expected stages of the contracting process.

#### Release tags

[Release tags](https://standard.open-contracting.org/latest/en/schema/codelists/#release-tag) indicate the stage of a contracting process to which a release is related.

Count the number of releases, for each release tag:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    tag,
    count(*)
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    tag
ORDER BY
    collection_id


#### Objects per stage

In OCDS, data is organized into objects, for each stage of a contracting process. Each compiled release has: at most one `Planning` object, at most one `Tender` object, any number of `Award` objects, and any number of `Contract` objects. Each `Contract` object has at most one `Implementation` object. As such, the number of `Award` objects can exceed the number of unique OCIDs, but the number of `Tender` objects can't.

Plot a count of objects per stage:

In [ ]:
%%sql objects_per_stage <<
SELECT
    CASE
        WHEN paths.path = 'contracts/implementation'
            THEN
                'implementation'
        ELSE
            paths.path
    END AS stage,
    CASE
        WHEN
            paths.path IN ('planning', 'tender', 'contracts/implementation')
            THEN
                greatest(field_counts.object_property, 0)
        ELSE
            greatest(field_counts.array_count, 0)
    END AS object_count
FROM (
    SELECT
        unnest(
            ARRAY[
                'planning',
                'tender',
                'awards',
                'contracts',
                'contracts/implementation'
            ]
        ) AS path
) AS paths
LEFT JOIN (
    SELECT *
    FROM
        field_counts
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
        AND path IN (
            'planning',
            'tender',
            'awards',
            'contracts',
            'contracts/implementation'
        )
) AS field_counts USING (path)


In [ ]:
plot_objects_per_stage(objects_per_stage)

### Date ranges


Use this section to check that the data covers the expected date range.

Calculate the earliest and latest `date`, `awards/date` and `contracts/dateSigned`:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    'release_date' AS date_type,
    min(date) AS min,
    max(date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'award_date' AS date_type,
    min(first_award_date) AS min,
    max(last_award_date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'contract_datesigned' AS date_type,
    min(first_contract_datesigned) AS min,
    max(last_contract_datesigned) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type
ORDER BY
    collection_id,
    release_type,
    date_type;


### Release date distribution

Use this section to check that releases are distributed as expected.

Plot the count of releases per month:

In [ ]:
%%sql release_dates <<
SELECT
    collection_id::text,
    release_type,
    date,
    count(*) AS release_count
FROM
    release_summary AS rs
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id,
    release_type,
    date
ORDER BY
    date ASC;


In [ ]:
# Resample by month
release_dates["date"] = release_dates["date"].dt.strftime("%Y-%m")
release_dates = (
    release_dates.groupby(["collection_id", "release_type", "date"]).agg({"release_count": "sum"}).reset_index()
)

plot_releases_by_month(release_dates)

### Extensions

Use this section to check which extensions the data uses.

List the extensions declared in the package metadata:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    jsonb_array_elements(package_data -> 'extensions') AS ocds_extension,
    count(*) AS count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND package_data IS NOT NULL
GROUP BY
    collection_id,
    release_type,
    ocds_extension
ORDER BY
    collection_id ASC,
    release_type ASC,
    count DESC;


## Check for structure and format errors

Kingfisher Collect reformats data sources as release packages or record packages. Check the `data_type` class attribute of the [spider](https://github.com/open-contracting/kingfisher-collect/tree/main/kingfisher_scrapy/spiders). If it is not 'release_package' or 'record_package', recommend to the publisher to package their data.

Kingfisher Process checks data against the OCDS schema using [lib-cove-ocds](https://github.com/open-contracting/lib-cove-ocds), same as the [OCDS Data Review Tool](https://review.standard.open-contracting.org). For release collections, Kingfisher Process stores check results in the `release_check` table. For record collections, Kingfisher Process stores check results in the `record_check` table.

### Confirm that checks are complete

If a crawl is scheduled using Kingfisher Collect, by default, Kingfisher Process performs structural checks. Checking data is the slowest step. For large collections, it is recommended to skip the `check` step or to collect only a sample. Otherwise, there can be a backlog of data to check.

Use this query to determine whether checks are complete for your collection(s).

If checks are in progress, you should wait for the checks to finish before running the queries in this section.

In [ ]:
%%sql
SELECT
    collection_id,
    'release' AS collection_type,
    CASE
        WHEN count(release.id) = count(release_check.id)
            THEN
                'complete'
        WHEN count(release_check.id) = 0
            THEN
                'not_started'
        ELSE
            'in_progress'
    END AS check_status,
    count(release_check.id)::text
    || '/'
    || count(release.id)::text AS check_progress
FROM
    release_check
RIGHT JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id
UNION
SELECT
    collection_id,
    'record' AS collection_type,
    CASE
        WHEN count(record.id) = count(record_check.id)
            THEN
                'complete'
        WHEN count(record_check.id) = 0
            THEN
                'not_started'
        ELSE
            'in_progress'
    END AS check_status,
    count(record_check.id)::text
    || '/'
    || count(record.id)::text AS check_progress
FROM
    record_check
RIGHT JOIN record ON record_check.record_id = record.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;


### Error summary

Summarize the errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_error_summary <<
WITH errors AS (
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        left(
            errors ->> 'description',
            49000
        ) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (
            PARTITION BY
                collection_id,
                errors ->> 'type',
                left(
                    errors ->> 'description',
                    49000
                )
        ) AS rownum
    FROM
        release_check AS rc
    CROSS JOIN
        jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    INNER JOIN release AS r ON rc.release_id = r.id
    WHERE
        collection_id IN :collection_ids
    UNION ALL
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        left(
            errors ->> 'description',
            49000
        ) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (
            PARTITION BY
                collection_id,
                errors ->> 'type',
                left(
                    errors ->> 'description',
                    49000
                )
        ) AS rownum
    FROM
        record_check AS rc
    CROSS JOIN
        jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    INNER JOIN record AS r ON rc.record_id = r.id
    WHERE
        collection_id IN :collection_ids
),

examples AS (
    SELECT
        collection_id,
        error_type,
        field,
        error,
        array_agg(ocid) AS example_ocids,
        array_agg(value) AS example_values
    FROM
        errors
    WHERE
        rownum <= 3
    GROUP BY
        collection_id,
        error_type,
        field,
        error
)

SELECT
    collection_id,
    error_type,
    errors.field,
    error,
    count(*) AS count,
    example_ocids,
    example_values
FROM
    errors
INNER JOIN examples USING (collection_id, error_type, error)
GROUP BY
    collection_id,
    error_type,
    errors.field,
    error,
    example_ocids,
    example_values;


In [ ]:
structure_and_format_error_summary

### Error details

List all errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_errors <<
SELECT
    collection_id,
    'release' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    left(
        errors ->> 'description',
        49000
    ) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    release_check AS rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
INNER JOIN release AS r ON rc.release_id = r.id
WHERE
    collection_id IN :collection_ids
UNION ALL
SELECT
    collection_id,
    'record' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    left(
        errors ->> 'description',
        49000
    ) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    record_check AS rc
CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
INNER JOIN record AS r ON rc.record_id = r.id
WHERE
    collection_id IN :collection_ids


In [ ]:
structure_and_format_errors

## Perform manual data quality checks

[Pelican](https://ocdsdeploy.readthedocs.io/en/latest/use/pelican.html) performs dozens of data quality checks that can be exported to Google Docs. It performs checks on compiled releases, not on individual releases, records, or package metadata.

Use this section to perform quality checks that require manual review, including:

* Package metadata
* Release metadata
* Free-text language (*can be added to Pelican if needed*)
* Change history
* Overfill
* Placeholder values
* Ground truth
* Identifier scheme preference
* Segmented field coverage

### Metadata

#### Package metadata

OCDS data must be published within either a [release package](https://standard.open-contracting.org/latest/en/schema/reference/#package-metadata) or a [record package](https://standard.open-contracting.org/latest/en/schema/records_reference/#package-metadata).

Use this section to check the values in the package metadata.

Look out for the following issues and report them to the publisher:

* Placeholder values
* Empty strings and objects
* Discrepancies in the package metadata between different releases

Display the package metadata for each collection:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    package_data -> 'version' AS ocds_version,
    package_data -> 'publisher' -> 'name' AS publisher_name,
    package_data -> 'publisher' -> 'name' -> 'scheme' AS publisher_scheme,
    package_data -> 'publisher' -> 'name' -> 'uid' AS publisher_uid,
    package_data -> 'publisher' -> 'name' -> 'uri' AS publisher_uri,
    package_data -> 'license' AS license,
    package_data -> 'publicationPolicy' AS publicationpolicy,
    count(*)
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND release_type <> 'compiled_release'
GROUP BY
    collection_id,
    release_type,
    publisher_name,
    publisher_scheme,
    publisher_uid,
    publisher_uri,
    license,
    publicationpolicy,
    ocds_version;


#### Release tags

> A release must be tagged to indicate whether it is about a planning process or a contracting process and, if it is about the latter, to indicate the stage of the contracting process to which it relates. (OCDS 1.2)

Use this section to check that release tags reflect the data included in each release.

Read the descriptions in the [codelist](https://standard.open-contracting.org/latest/en/schema/codelists/#release-tagf) to understand which sections can be provided for each tag.

Remember that releases can repeat information from previous releases.

Count the number of times a section is published, for each release tag.

Note that this check only counts whether the section exists, not whether it contains any fields or objects, so the results may include empty objects (e.g. `planning`) and arrays (e.g. `awards`).

In [ ]:
%%sql release_tag_section_summary <<
WITH contract_implementation AS (
    SELECT
        cs.collection_id,
        cs.release_type,
        tag,
        count(contract -> 'implementation') AS contract_implementation
    FROM
        contracts_summary AS cs
    LEFT JOIN release_summary USING (id)
    GROUP BY
        cs.collection_id, cs.release_type, tag
),

sections AS (
    SELECT
        collection_id,
        release_type,
        tag,
        count(*) AS release_count,
        count(release -> 'planning') AS planning,
        count(release -> 'tender') AS tender,
        count(release -> 'awards') AS award,
        count(release -> 'contracts') AS contract
    FROM
        release_summary
    GROUP BY
        collection_id,
        release_type,
        tag
)

SELECT
    collection_id,
    release_type,
    sections.tag,
    release_count,
    planning,
    tender,
    award,
    contract,
    contract_implementation
FROM
    sections
LEFT JOIN contract_implementation USING (collection_id, release_type, tag);


In [ ]:
release_tag_section_summary

#### Release date

Use this section to check that all releases do not share the same date.

For each collection and release type, generate a [frequency table](https://en.wikipedia.org/wiki/Frequency_distribution) for release dates and report the top 5 most frequent dates:

In [ ]:
%%sql
WITH date_frequency AS (
    SELECT
        collection_id,
        release_type,
        date,
        count(*) AS release_count,
        row_number() OVER (
            PARTITION BY
                collection_id,
                release_type
            ORDER BY count(*) DESC
        ) AS row_number
    FROM
        release_summary
    GROUP BY
        collection_id,
        release_type,
        date
)

SELECT
    collection_id,
    release_type,
    date,
    release_count
FROM
    date_frequency
WHERE
    row_number <= 5
ORDER BY
    collection_id ASC,
    release_type ASC,
    release_count DESC;


#### Language

> The default language of the data, from the open language codelist. A BCP47 language tag is allowed, if there is a user need for the additional information.

Use this section to check that the code declared in `language` reflects the language used in free-text fields in the data.

List the language codes used, with an example release for each language.

In [ ]:
%%sql
SELECT DISTINCT ON (
    collection_id, release_type,
    language)
    collection_id,
    release_type,
    language,
    release AS example_release
FROM
    release_summary
ORDER BY
    collection_id,
    release_type,
    language;


### Change history

OCDS supports the publication of a change history, using [releases and records](https://standard.open-contracting.org/latest/en/primer/releases_and_records/).

Fully implemented, releases and records can be used to publish the following for each contracting process:

* Multiple releases, one for each change or update to the contracting process
* A single record, containing:
  * `releases`: an index of releases for the contracting process
  * optionally, a `compiledRelease`: the latest version of the data about the contracting process
  * optionally, a `versionedRelease`: a change history for each field

However, many publishers use the ['easy releases'](https://standard.open-contracting.org/latest/en/guidance/build/easy_releases/) approach: publish a single release per contracting process with the latest version of the data about the contracting process.

Use this section to understand the approach used by the publisher.

#### Multiple releases per contracting process

Use this section to:

* check if there are multiple releases per contracting process
* check the distribution of releases per contracting process
* examine examples of contracting processes with multiple releases

Calculate statistics on the minimum, maximum, average and standard deviation of releases per contracting process.

In [ ]:
%%sql
WITH release_counts AS (
    SELECT
        collection_id,
        release_type,
        ocid,
        count(*) AS release_count
    FROM
        release_summary AS rs
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id,
        release_type,
        ocid
)

SELECT
    collection_id,
    release_type,
    min(release_count) AS min_releases_per_ocid,
    max(release_count) AS max_releases_per_ocid,
    round(avg(release_count), 2) AS avg_releases_per_ocid,
    round(stddev(release_count), 2) AS sd_releases_per_ocid
FROM
    release_counts
GROUP BY
    collection_id,
    release_type;


Count the number of contracting processes, for each observed number of releases:

In [ ]:
%%sql release_count_summary <<
WITH release_counts AS (
    SELECT
        collection_id,
        release_type,
        ocid,
        count(*) AS release_count
    FROM
        release_summary AS rs
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('release', 'embedded_release')
    GROUP BY
        collection_id,
        release_type,
        ocid
)

SELECT
    collection_id,
    release_type,
    release_count,
    count(*) AS contracting_processes
FROM
    release_counts
GROUP BY
    collection_id,
    release_type,
    release_count;


In [ ]:
release_count_summary

Plot the distribution of releases per contracting process:

In [ ]:
%%sql release_counts <<
WITH release_counts AS (
    SELECT
        collection_id,
        release_type,
        ocid,
        count(*) AS release_count
    FROM
        release_summary AS rs
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('release', 'embedded_release')
    GROUP BY
        collection_id,
        release_type,
        ocid
)

SELECT
    collection_id,
    release_type,
    release_count,
    count(*) AS ocid_count
FROM
    release_counts
GROUP BY
    collection_id,
    release_type,
    release_count;


In [ ]:
plot_release_count(release_counts)

Create a release package containing the top 5 contracting processes with the most releases.

Specific things to check include:

* Does the `date` field differ between releases?
* Does the `tag` field differ between releases?

Also check for differences in which fields are provided for each release and for differences in the values of fields.


In [ ]:
%%sql multiple_release_examples <<
WITH ranked_ocids AS (
    SELECT
        collection_id,
        release_type,
        ocid,
        count(*),
        row_number() OVER (
            PARTITION BY
                collection_id,
                release_type
            ORDER BY count(*) DESC
        ) AS row_number
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('release', 'embedded_release')
    GROUP BY
        collection_id,
        release_type,
        ocid
)

SELECT jsonb_build_object('releases', jsonb_agg(release)) AS release_package
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND release_type IN ('release', 'embedded_release')
    AND ocid IN (
        SELECT ocid
        FROM
            ranked_ocids
        WHERE
            row_number <= 5
    );


In [ ]:
render_json(multiple_release_examples["release_package"][0])

To ease review, uncomment the following cell to convert the release package to a Google Sheet:

In [ ]:
# spreadsheet_name = ''
# save_dataframe_to_spreadsheet(multiple_release_examples, f'{spreadsheet_name}_multiple_releases')

#### Static release ID

The release identifier must be updated when the information about a contracting process changes.

A common error is to set the release ID to the same value as the `ocid`, to set it to a subset of the `ocid`, and to neglect to update it.

Use this section to check that the release ID differs from the `ocid`.

List the releases where `id` and `ocid` have the same value:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    ocid,
    release_id
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND (
        ocid = release_id
        OR ocid ILIKE '%%' || release_id || '%%'
    )


#### Duplicate release ID

In [ ]:
%%sql
SELECT
    ocid,
    release_id,
    count(*) AS release_count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
GROUP BY
    ocid,
    release_id
HAVING
    count(*) > 1


Export to Google Sheets the 5 release IDs with the most duplicates, and review them to determine whether the full release is duplicated or only the release ID.

In [ ]:
%%sql duplicate_release_ids <<
WITH release_ids AS (
    SELECT
        collection_id,
        ocid,
        release_type,
        release_id,
        count(*) AS release_count,
        row_number() OVER (
            PARTITION BY
                collection_id,
                release_type
            ORDER BY count(*) DESC
        ) AS row_number
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
    GROUP BY
        collection_id,
        ocid,
        release_type,
        release_id
    HAVING
        count(*) > 1
)

SELECT jsonb_build_object('releases', jsonb_agg(release)) AS release_package
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND release_type IN ('release', 'embedded_release')
    AND ocid IN (
        SELECT ocid
        FROM
            release_ids
        WHERE
            row_number <= 5
    );


In [ ]:
render_json(duplicate_release_ids["release_package"][0])

In [ ]:
# save_dataframe_to_spreadsheet(duplicate_release_ids, f'{spreadsheet_name}_duplicate_release_ids')

### Overfill

In a whole dataset, we expect there to be some differences between the values, items and dates listed in the tender, award and contract sections of OCDS.

In an effort to publish as many field as possible, publishers sometimes ignore semantics and map one field from their data source to several fields in OCDS, known as overfill.

Use this section to identify instances of overfill.

#### Awards and contracts

Use this section to check if there are any differences between the following fields in the award and contract sections:

* `awards/date` and `contracts/dateSigned`
* `awards/value` and `contracts/value`
* `awards/items` and `contracts/items`
* `awards/contractPeriod` and `contracts/period`
* `award/documents` and `contracts/documents`

In [ ]:
%%sql
SELECT
    contracts_summary.collection_id,
    contracts_summary.release_type,
    coalesce(
        awards_summary.date = contracts_summary.datesigned,
        FALSE
    ) AS date_match,
    coalesce(
        (awards_summary.value_amount = contracts_summary.value_amount)
        AND (awards_summary.value_currency = contracts_summary.value_currency),
        FALSE
    ) AS value_match,
    coalesce((
        awards_summary.contractperiod_startdate
        = contracts_summary.period_startdate
    )
    AND (
        awards_summary.contractperiod_enddate
        = contracts_summary.period_startdate
    ), FALSE) AS period_match,
    coalesce(
        awards_summary.award ->> 'documents'
        = contracts_summary.contract ->> 'documents',
        FALSE
    ) AS documents_match,
    count(contracts_summary.id) AS contract_count
FROM
    contracts_summary
INNER JOIN awards_summary
    ON
        contracts_summary.id = awards_summary.id
        AND contracts_summary.awardid = awards_summary.award_id
WHERE
    contracts_summary.collection_id IN :collection_ids
    AND contracts_summary.release_type IN ('record', 'compiled_release')
GROUP BY
    contracts_summary.collection_id,
    contracts_summary.release_type,
    date_match,
    value_match,
    period_match,
    documents_match
ORDER BY
    contracts_summary.collection_id ASC,
    contracts_summary.release_type ASC,
    contract_count DESC


#### Items

Items are attached to the tender, award and contract sections of a release, so that users can see if there were any changes to the items being procured during the contracting process.

Use this section to check for differences between the items attached to the tender, award and contract sections.

In [ ]:
%%sql
SELECT
    tender_summary.collection_id,
    tender_summary.release_type,
    coalesce(
        contracts_summary.contract -> 'items'
        = awards_summary.award -> 'items',
        FALSE
    ) AS award_contract_match,
    count(contracts_summary.id) AS contracts_count,
    coalesce(
        awards_summary.award -> 'items'
        = tender_summary.tender -> 'items',
        FALSE
    ) AS tender_award_match,
    count(awards_summary.id) AS awards_count
FROM
    tender_summary
INNER JOIN awards_summary USING (id)
LEFT JOIN contracts_summary
    ON
        awards_summary.id = contracts_summary.id
        AND awards_summary.award_id = contracts_summary.awardid
WHERE
    tender_summary.collection_id IN :collection_ids
    AND tender_summary.release_type IN ('record', 'compiled_release')
GROUP BY
    tender_summary.collection_id,
    tender_summary.release_type,
    award_contract_match,
    tender_award_match;


### Placeholder values

Use this section to check for placeholder values.

Manually review the example release to identify placeholder values, e.g. 'n/a', 'test', '1970-01-01T00:00:00Z' etc.

Get an example release:

In [ ]:
%%sql example_releases <<
WITH examples AS (
    SELECT DISTINCT ON (
        collection_id,
        release_type)
        collection_id,
        release_type,
        release
    FROM
        release_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('release', 'embedded_release')
    ORDER BY
        collection_id,
        release_type,
        random()
)

SELECT jsonb_build_object('releases', jsonb_agg(release)) AS release_package
FROM
    examples


In [ ]:
render_json(example_releases["release_package"][0])

### Ground truth

#### Organization identifiers

Publishers should collect and publish [organization identifiers](https://standard.open-contracting.org/latest/en/schema/identifiers/#organization-ids).

Use this section to check for invalid or incorrect organization identifiers. (Pelican checks schemes, not identifiers.)

For each organization identifier:

1. Look up the `scheme` in [org-id.guide](http://org-id.guide/) and follow the guidance to look up the organization identifiers in the register.
1. Check that the identifier exists in the register.

Select a random sample of 3 identifiers for each organization identifier scheme:

In [ ]:
%%sql organization_identifiers <<
SELECT
    collection_id,
    release_type,
    party ->> 'name' AS name,
    party -> 'identifier' ->> 'legalName' AS legalname,
    roles,
    party -> 'identifier' ->> 'scheme' AS scheme,
    party -> 'identifier' ->> 'id' AS id,
    ocid
FROM
    parties_summary
WHERE
    collection_id IN :collection_ids;


organization_identifiers.groupby(["collection_id", "release_type", "scheme"]).sample(n=3)

### Item classifications

Use this section to check whether the data includes item classifications.

In [ ]:
%%sql
WITH items AS (
    SELECT
        collection_id,
        release_type,
        'tender' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        tender_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
    UNION ALL
    SELECT
        collection_id,
        release_type,
        'award' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        award_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
    UNION ALL
    SELECT
        collection_id,
        release_type,
        'contract' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        contract_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
)

SELECT
    collection_id,
    release_type,
    stage,
    count(*) AS item_count,
    scheme
FROM
    items
GROUP BY
    collection_id,
    release_type,
    scheme,
    stage
ORDER BY stage;


Select a random sample of 3 identifiers for each item identifier scheme:

In [ ]:
%%sql item_identifiers <<
WITH items AS (
    SELECT
        collection_id,
        release_type,
        'tender' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        tender_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
    UNION ALL
    SELECT
        collection_id,
        release_type,
        'award' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        award_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
    UNION ALL
    SELECT
        collection_id,
        release_type,
        'contract' AS stage,
        item -> 'classification' ->> 'id' AS id,
        item -> 'classification' ->> 'scheme' AS scheme
    FROM
        contract_items_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type = 'compiled_release'
)

SELECT *
FROM
    items
WHERE id IS NOT NULL;


In [ ]:
item_identifiers.groupby(["collection_id", "release_type", "stage", "scheme"]).sample(n=3)

#### Document metadata

Use this section to check that document metadata is accurate.

Retrieve the document from the `url` and check that each metadata field accurate reflects the actual document.





Get a random document:

In [ ]:
%%sql
WITH documents AS (
    SELECT
        collection_id,
        release_type,
        'planning' AS section,
        ocid,
        document
    FROM
        planning_documents_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
    UNION
    SELECT
        collection_id,
        release_type,
        'tender' AS section,
        ocid,
        document
    FROM
        tender_documents_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
    UNION
    SELECT
        collection_id,
        release_type,
        'awards' AS section,
        ocid,
        document
    FROM
        award_documents_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
    UNION
    SELECT
        collection_id,
        release_type,
        'contracts' AS section,
        ocid,
        document
    FROM
        contract_documents_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
    UNION
    SELECT
        collection_id,
        release_type,
        ocid,
        'implementation' AS section,
        document
    FROM
        contract_implementation_documents_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
)

SELECT DISTINCT ON (collection_id, release_type)
    collection_id,
    release_type,
    section,
    document ->> 'id' AS id,
    document ->> 'documentType' AS documenttype,
    document ->> 'title' AS title,
    document ->> 'description' AS description,
    document ->> 'url' AS url,
    document ->> 'datePublished' AS datepublished,
    document ->> 'dateModified' AS datemodified,
    document ->> 'format' AS format,
    document ->> 'language' AS
    language
FROM
    documents
ORDER BY
    collection_id,
    release_type;


### Identifier scheme preference

Publishers sometimes use a domestic register as the `.scheme` for foreign-registered companies. It is preferred to use an international register.

Use this section to check whether an international organization identifier scheme is provided for foreign-registered companies.

Set the `country` variable to the name of the country for the publisher before running the query.

For each organization identifier:

1. Look up the `scheme` in [org-id.guide](http://org-id.guide/) and follow the guidance to look up the organization identifiers in the register.
1. Check that the identifier exists in the register.

In [ ]:
country = "Paraguay"

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    name,
    scheme,
    id,
    legalname,
    country,
    roles
FROM (
    SELECT DISTINCT
        collection_id,
        release_type,
        party ->> 'name' AS name,
        party -> 'identifier' ->> 'scheme' AS scheme,
        party -> 'identifier' ->> 'id' AS id,
        party -> 'identifier' ->> 'legalName' AS legalname,
        party -> 'address' ->> 'country' AS country,
        roles,
        rank()
            OVER (
                PARTITION BY
                    collection_id,
                    release_id,
                    party -> 'identifier' ->> 'scheme'
                ORDER BY random()
            )
    FROM parties_summary
    WHERE
        collection_id IN :collection_ids
        AND release_type IN ('record', 'compiled_release')
        AND party -> 'address' ->> 'country' NOT ILIKE :country
) AS identifiers
WHERE
    rank <= 3
ORDER BY
    scheme;


### Coverage

Coverage is covered by Pelican. This section segments field coverage for priority fields (Pelican does not segment by, e.g., party role).


Use this section to check whether the data includes key fields.

#### Organization identifiers

Use this section to check whether the data includes organization identifiers for buyers, procuring entities, suppliers and tenderers.

Calculate the coverage of `parties/identifier/id` and `parties/identifier/scheme`, grouped by `parties/role`:

In [ ]:
%%sql
SELECT
    collection_id,
    release_type,
    CASE
        WHEN roles @> '["buyer"]'::jsonb
            THEN
                'buyer'
        WHEN roles @> '["procuringEntity"]'::jsonb
            THEN
                'procuringEntity'
        WHEN roles @> '["supplier"]'::jsonb
            THEN
                'supplier'
        WHEN roles @> '["tenderer"]'::jsonb
            THEN
                'tenderer'
        ELSE
            'other'
    END AS role,
    count(*) AS party_count,
    round(sum(
        CASE
            WHEN party -> 'identifier' ->> 'id' IS NOT NULL
                THEN
                    1
            ELSE
                0
        END
    )::numeric / count(*), 2) AS id_coverage,
    round(sum(
        CASE
            WHEN party -> 'identifier' ->> 'scheme' IS NOT NULL
                THEN
                    1
            ELSE
                0
        END
    )::numeric / count(*), 2) AS scheme_coverage
FROM
    parties_summary
WHERE
    collection_id IN :collection_ids
    AND release_type = 'compiled_release'
GROUP BY
    collection_id,
    release_type,
    role;


### OCID modeling

Use this section to check if the OCID is being modeled as expected

#### Number of tenders per ocid


Use this section to check there is always only one tender per ocid.

If the data contains planning data and multiple tenders per ocid, it might indicate that the ocid is being assigned in the planning stage, and the planning can have more than one tender, for example, due to unsuccessful tenders.

In [ ]:
%%sql
SELECT
    ocid,
    count(DISTINCT data -> 'tender' ->> 'id') AS cnt
FROM data
INNER JOIN release ON data.id = data_id
WHERE collection_id IN :collection_ids
GROUP BY ocid
ORDER BY cnt DESC;
